In [36]:
import os
import re

import keras
import urllib
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras import metrics
from keras import backend as K
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping, TensorBoard
from keras import regularizers
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras import backend as K


%matplotlib inline
sns.set_style('white')
plt.style.use('bmh')

In [5]:
data_dir = ""

In [6]:
X = np.load('X_array.npy'.format(data_dir))
# Shape should be (4722, 138, 92, 3)
print(X.shape)

(4722, 138, 92, 3)


In [7]:
Y = np.load('Y_array.npy'.format(data_dir))
# Shape should be (4722, 20)
print(Y.shape)

(4722, 20)


In [34]:
batch_size = 126
n_epochs = 10
input_shape = (138,92,3)
num_classes= 20

In [ ]:
# Super simple CNN
model = Sequential()

for n_conv_layers in [3]:
    print(n_conv_layers)
    log_dir_string = "cv"

    K.clear_session()

    # Input layer
    model.add(Conv2D(80, activation='relu', 
                     kernel_size=(5, 5),
                     use_bias=False,
                     data_format = 'channels_last',
                     input_shape=(138,92,3)))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    if n_conv_layers == 1:
        log_dir_string += "_1conv"
    elif n_conv_layers == 2:
        log_dir_string += "_2conv"
        model.add(Conv2D(16, 
                         kernel_size=(5, 5), 
                         activation='relu',
                         input_shape=(138,92,3),
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))    
    elif n_conv_layers == 3:
        log_dir_string += "_3conv"
        model.add(Conv2D(16, 
                         kernel_size=(5, 5),
                         activation='relu',
                         input_shape=(138,92,3),
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))  

        model.add(Conv2D(32, 
                         kernel_size=(5, 5),
                         activation='relu',
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
    else:
        log_dir_string += "_4conv"
        model.add(Conv2D(16, 
                         kernel_size=(5, 5), 
                         activation='relu',
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))  

        model.add(Conv2D(32, 
                         kernel_size=(5, 5), 
                         activation='relu',
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, 
                         kernel_size=(5, 5), 
                         activation='relu',
                         use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(20, activation='sigmoid'))

    sgd = SGD(lr=0.1, momentum=0.9)
    
    model.compile(optimizer=sgd,
                  loss=['binary_crossentropy'], 
                  metrics=['mae', metrics.categorical_accuracy]
                 )

    tensorboard = TensorBoard(log_dir='./logs/cv/'+log_dir_string,
                                      histogram_freq=1, 
                                      write_graph=True, 
                                      write_images=True
                                     )

    early_stopping_monitor = EarlyStopping(patience = 2)

    history = model.fit(X, Y,
                        validation_split = 0.2,
                        batch_size=batch_size,
                        epochs = n_epochs,
                        class_weight = 'auto',
                        callbacks = [early_stopping_monitor,tensorboard],
                        verbose=1)

In [39]:
for n_conv_layers in [1,2]:
    for n_dense_layers in [1,2]:
        log_dir_string = "cv"
        K.clear_session()
        model = Sequential()
        
        model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape, use_bias=False))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        if n_conv_layers == 1:
            log_dir_string += "_1conv"
        else:
            model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape, use_bias=False))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            log_dir_string += "_2conv"
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        
        if n_dense_layers == 1:
            log_dir_string += "_1fc"
        else:
            model.add(Dense(64, activation='relu'))
            log_dir_string += "_2fc"
        
        model.add(Dense(num_classes, activation='sigmoid'))

        adam = Adam(lr=0.001)
        
        model.compile(optimizer=adam,
              loss=['binary_crossentropy'], 
              metrics=['mae', metrics.categorical_accuracy]
             )

        
                # we need a callback to save information for tensorboard visualizations
        tensorboard = TensorBoard(log_dir='./logs-simple/cv/'+log_dir_string,
                                                  histogram_freq=1, 
                                                  write_graph=True, 
                                                  write_images=False
                                                 )

        history = model.fit(X, Y,
                            validation_split = 0.2,
                            batch_size=batch_size,
                            epochs = n_epochs,
                            class_weight = 'auto',
                            callbacks = [tensorboard],
                            verbose=1)


Train on 3777 samples, validate on 945 samples
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
Epoch 1/10
3777/3777 [==============================] - 7s - loss: 0.3833 - mean_absolute_error: 0.2018 - categorical_accuracy: 0.2597 - val_loss: 0.3053 - val_mean_absolute_error: 0.1950 - val_categorical_accuracy: 0.2952
Epoch 2/10
3777/3777 [==============================] - 6s - loss: 0.2894 - mean_absolute_error: 0.1679 - categorical_accuracy: 0.3320 - val_loss: 0.2917 - val_mean_absolute_error: 0.1679 - val_categorical_accuracy: 0.3418
Epoch 3/10
3777/3777 [==============================] - 6s - lo

3777/3777 [==============================] - 9s - loss: 0.2308 - mean_absolute_error: 0.1387 - categorical_accuracy: 0.4464 - val_loss: 0.3102 - val_mean_absolute_error: 0.1566 - val_categorical_accuracy: 0.2889
Epoch 10/10
3777/3777 [==============================] - 9s - loss: 0.2149 - mean_absolute_error: 0.1308 - categorical_accuracy: 0.4668 - val_loss: 0.3260 - val_mean_absolute_error: 0.1493 - val_categorical_accuracy: 0.2910
Train on 3777 samples, validate on 945 samples
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense

In [ ]:
plt.plot(history.history['acc'])
plt.xlabel("epoch")
plt.ylabel("accuracy")